In [25]:
import re
import numpy as np
import html

from fastai.text import * 

chunksize=24000

In [3]:
re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

In [86]:
def get_texts(df, n_lbls=1):
    print(range(n_lbls))
    labels = df.iloc[:,range(n_lbls)].values.astype(np.int32)
    texts = '\n{BOS} {FLD} 1 ' + df[n_lbls].astype(str)
    for i in range(n_lbls+1, len(df.columns)): texts += ' {FLD} {i-n_lbls} ' + df[i].astype(str)
    texts = list(texts.apply(fixup).values)

    tok = Tokenizer().proc_all_mp(partition_by_cores(texts))
    return tok, list(labels)

In [78]:
def get_all(df, n_lbls):
    tok, labels = [], []
    for i, r in enumerate(df):
        print(i)
        tok_, labels_ = get_texts(r, n_lbls)
        tok += tok_;
        labels += labels_
    return tok, labels

In [81]:
import pandas as pd

#from google.colab import drive
df_trn = pd.read_csv('data/train.csv', header=None, chunksize=chunksize)
df_val = pd.read_csv('data/test.csv', header=None, chunksize=chunksize)

In [82]:
#df_trn.info()

In [87]:
tok_trn, trn_labels = get_all(df_trn, 1)
tok_val, val_labels = get_all(df_val, 1)

In [88]:
LM_PATH = Path('data/')
LM_PATH.mkdir(exist_ok=True)

(LM_PATH/'tmp').mkdir(exist_ok=True)

In [89]:
np.save(LM_PATH/'tmp'/'tok_trn.npy', tok_trn)
np.save(LM_PATH/'tmp'/'tok_val.npy', tok_val)

In [90]:
tok_trn = np.load(LM_PATH/'tmp'/'tok_trn.npy')
tok_val = np.load(LM_PATH/'tmp'/'tok_val.npy')

In [91]:
freq = Counter(p for o in tok_trn for p in o)
freq.most_common(25)

[]

In [92]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 

--2019-01-13 20:52:35--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolviendo ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Conectando con ai.stanford.edu (ai.stanford.edu)[171.64.68.10]:80... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 84125825 (80M) [application/x-gzip]
Grabando a: “aclImdb_v1.tar.gz”

aclImdb_v1.tar.gz   100%[===================>]  80,23M  1,57MB/s    in 60s     

2019-01-13 20:53:36 (1,33 MB/s) - “aclImdb_v1.tar.gz” guardado [84125825/84125825]



In [93]:
!tar -xf aclImdb_v1.tar.gz

In [95]:
!mv aclImdb data/

In [97]:
!rm aclImdb_v1.tar.gz

In [3]:
from fastai.text import * 

import html

In [4]:
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag

PATH=Path('data/aclImdb/')

In [5]:
CLAS_PATH=Path('data/imdb_clas/')
CLAS_PATH.mkdir(exist_ok=True)

LM_PATH=Path('data/imdb_lm/')
LM_PATH.mkdir(exist_ok=True)

In [6]:
CLASSES = ['neg', 'pos', 'unsup']

def get_texts(path):
    texts,labels = [],[]
    for idx,label in enumerate(CLASSES):
        for fname in (path/label).glob('*.*'):
            texts.append(fname.open('r', encoding='utf-8').read())
            labels.append(idx)
        sys.stdout.write('\r{:5.2f}%'.format(100*(idx+1)/3))
    sys.stdout.write('\rDone     \n\n')
    return np.array(texts),np.array(labels)

trn_texts,trn_labels = get_texts(PATH/'train')
val_texts,val_labels = get_texts(PATH/'test')

Done     

Done     



In [7]:
len(trn_texts),len(val_texts)

(75000, 25000)

In [8]:
col_names = ['labels','text']

In [9]:
np.random.seed(42)
trn_idx = np.random.permutation(len(trn_texts))
val_idx = np.random.permutation(len(val_texts))

In [10]:
trn_texts = trn_texts[trn_idx]
val_texts = val_texts[val_idx]

trn_labels = trn_labels[trn_idx]
val_labels = val_labels[val_idx]

In [11]:
df_trn = pd.DataFrame({'text':trn_texts, 'labels':trn_labels}, columns=col_names)
df_val = pd.DataFrame({'text':val_texts, 'labels':val_labels}, columns=col_names)

In [13]:
df_trn[df_trn['labels']!=2].to_csv(CLAS_PATH/'train.csv', header=False, index=False)
df_val.to_csv(CLAS_PATH/'test.csv', header=False, index=False)

(CLAS_PATH/'classes.txt').open('w', encoding='utf-8').writelines('{o}\n' for o in CLASSES)

In [17]:
#trn_texts,val_texts = sklearn.model_selection.train_test_split(np.concatenate([trn_texts,val_texts]), test_size=0.1)

In [15]:
len(trn_texts), len(val_texts)

(75000, 25000)

In [16]:
df_trn = pd.DataFrame({'text':trn_texts, 'labels':[0]*len(trn_texts)}, columns=col_names)
df_val = pd.DataFrame({'text':val_texts, 'labels':[0]*len(val_texts)}, columns=col_names)

df_trn.to_csv(LM_PATH/'train.csv', header=False, index=False)
df_val.to_csv(LM_PATH/'test.csv', header=False, index=False)